<a href="https://colab.research.google.com/github/shirinrafia/https-github.com-RishubHighIQ-InstaBase_python_trainer/blob/main/Python_Trainer_Notebook_shirin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import tensorflow as tf
from PIL import Image
import numpy as np
from matplotlib.cbook import deprecated


class ImageClassification:
   
    def __init__(self):
        self.__modelType = ""
        self.modelPath = ""
        self.__modelLoaded = False
        self.__model_collection = []
        self.__input_image_size = 224
    
    def setModelPath(self, model_path):
        """
        'setModelPath()' function is required and is used to set the file path to the model adopted from the list of the
        available 4 model types. The model path must correspond to the model type set for the classification instance object.
        :param model_path:
        :return:
        """
        self.modelPath = model_path

    def setModelTypeAsSqueezeNet(self):
        raise ValueError("ImageAI no longer support SqueezeNet. You can use MobileNetV2 instead by downloading the MobileNetV2 model and call the function 'setModelTypeAsMobileNetV2'")

    def setModelTypeAsMobileNetV2(self):
        """
        'setModelTypeAsMobileNetV2()' is used to set the model type to the MobileNetV2 model
        for the classification instance object .
        :return:
        """
        self.__modelType = "mobilenetv2"

    @deprecated(since="2.1.6", message="'.setModelTypeAsResNet()' has been deprecated! Please use 'setModelTypeAsResNet50()' instead.")
    def setModelTypeAsResNet(self):
        return self.setModelTypeAsResNet50()

    def setModelTypeAsResNet50(self):
        """
         'setModelTypeAsResNet50()' is used to set the model type to the ResNet50 model
                for the classification instance object .
        :return:
        """
        self.__modelType = "resnet50"

    @deprecated(since="2.1.6", message="'.setModelTypeAsDenseNet()' has been deprecated! Please use 'setModelTypeAsDenseNet121()' instead.")
    def setModelTypeAsDenseNet(self):
        return self.setModelTypeAsDenseNet121()

    def setModelTypeAsDenseNet121(self):
        """
         'setModelTypeAsDenseNet121()' is used to set the model type to the DenseNet121 model
                for the classification instance object .
        :return:
        """
        self.__modelType = "densenet121"

    def setModelTypeAsInceptionV3(self):
        """
         'setModelTypeAsInceptionV3()' is used to set the model type to the InceptionV3 model
                for the classification instance object .
        :return:
        """
        self.__modelType = "inceptionv3"

    def loadModel(self, classification_speed="normal"):
        """
        'loadModel()' function is used to load the model structure into the program from the file path defined
        in the setModelPath() function. This function receives an optional value which is "classification_speed".
        The value is used to reduce the time it takes to classify an image, down to about 50% of the normal time,
        with just slight changes or drop in classification accuracy, depending on the nature of the image.
        * classification_speed (optional); Acceptable values are "normal", "fast", "faster" and "fastest"
        :param classification_speed :
        :return:
        """

        if(classification_speed=="normal"):
            self.__input_image_size = 224
        elif(classification_speed=="fast"):
            self.__input_image_size = 160
        elif(classification_speed=="faster"):
            self.__input_image_size = 120
        elif (classification_speed == "fastest"):
            self.__input_image_size = 100

        if (self.__modelLoaded == False):

            if(self.__modelType == "" ):
                raise ValueError("You must set a valid model type before loading the model.")

            elif(self.__modelType == "mobilenetv2"):
                model = tf.keras.applications.MobileNetV2(input_shape=(self.__input_image_size, self.__input_image_size, 3), weights=None, classes = 1000 )
                model.load_weights(self.modelPath)
                self.__model_collection.append(model)
                self.__modelLoaded = True
                try:
                    None
                except:
                    raise ValueError("An error occured. Ensure your model file is a MobileNetV2 Model and is located in the path {}".format(self.modelPath))

            elif(self.__modelType == "resnet50"):
                try:
                    model = tf.keras.applications.ResNet50(input_shape=(self.__input_image_size, self.__input_image_size, 3), weights=None, classes = 1000 )
                    model.load_weights(self.modelPath)
                    self.__model_collection.append(model)
                    self.__modelLoaded = True
                except Exception as e:
                    raise ValueError("An error occured. Ensure your model file is a ResNet50 Model and is located in the path {}".format(self.modelPath))

            elif (self.__modelType == "densenet121"):
                try:
                    model = tf.keras.applications.DenseNet121(input_shape=(self.__input_image_size, self.__input_image_size, 3), weights=None, classes = 1000 )
                    model.load_weights(self.modelPath)
                    self.__model_collection.append(model)
                    self.__modelLoaded = True
                except:
                    raise ValueError("An error occured. Ensure your model file is a DenseNet121 Model and is located in the path {}".format(self.modelPath))

            elif (self.__modelType == "inceptionv3"):
                try:
                    model = tf.keras.applications.InceptionV3(input_shape=(self.__input_image_size, self.__input_image_size, 3), weights=None, classes = 1000 )
                    model.load_weights(self.modelPath)
                    self.__model_collection.append(model)
                    self.__modelLoaded = True
                except:
                    raise ValueError("An error occured. Ensure your model file is in {}".format(self.modelPath))


    def classifyImage(self, image_input, result_count=5, input_type="file"):
        
        classification_results = []
        classification_probabilities = []
        if (self.__modelLoaded == False):
            raise ValueError("You must call the loadModel() function before making classification.")

        else:
            if (input_type == "file"):
                try:
                    image_to_predict = tf.keras.preprocessing.image.load_img(image_input, target_size=(self.__input_image_size, self.__input_image_size))
                    image_to_predict = tf.keras.preprocessing.image.img_to_array(image_to_predict, data_format="channels_last")
                    image_to_predict = np.expand_dims(image_to_predict, axis=0)
                except:
                    raise ValueError("You have set a path to an invalid image file.")
            elif (input_type == "array"):
                try:
                    image_input = Image.fromarray(np.uint8(image_input))
                    image_input = image_input.resize((self.__input_image_size, self.__input_image_size))
                    image_input = np.expand_dims(image_input, axis=0)
                    image_to_predict = image_input.copy()
                    image_to_predict = np.asarray(image_to_predict, dtype=np.float64)
                except:
                    raise ValueError("You have parsed in a wrong numpy array for the image")
            elif (input_type == "stream"):
                try:
                    image_input = Image.open(image_input)
                    image_input = image_input.resize((self.__input_image_size, self.__input_image_size))
                    image_input = np.expand_dims(image_input, axis=0)
                    image_to_predict = image_input.copy()
                    image_to_predict = np.asarray(image_to_predict, dtype=np.float64)
                    
                except:
                    raise ValueError("You have parsed in a wrong stream for the image")
            
            if (self.__modelType == "mobilenetv2"):
                image_to_predict = tf.keras.applications.mobilenet_v2.preprocess_input(image_to_predict)
            elif (self.__modelType == "densenet121"):
                image_to_predict = tf.keras.applications.densenet.preprocess_input(image_to_predict)
            elif (self.__modelType == "inceptionv3"):
                image_to_predict = tf.keras.applications.inception_v3.preprocess_input(image_to_predict)

            try:
                model = self.__model_collection[0]
                prediction = model.predict(image_to_predict, steps=1)

                if (self.__modelType == "mobilenetv2"):
                    predictiondata = tf.keras.applications.mobilenet_v2.decode_predictions(prediction, top=int(result_count))
                elif (self.__modelType == "resnet50"):
                    predictiondata = tf.keras.applications.resnet50.decode_predictions(prediction, top=int(result_count))
                elif (self.__modelType == "inceptionv3"):
                    predictiondata = tf.keras.applications.inception_v3.decode_predictions(prediction, top=int(result_count))
                elif (self.__modelType == "densenet121"):
                    predictiondata = tf.keras.applications.densenet.decode_predictions(prediction, top=int(result_count))

                

                for results in predictiondata:
                    for result in results:
                        classification_results.append(str(result[1]))
                        classification_probabilities.append(result[2] * 100)
            except:
                raise ValueError("An error occured! Try again.")

            return classification_results, classification_probabilities
                

    @deprecated(since="2.1.6", message="'.predictImage()' has been deprecated! Please use 'classifyImage()' instead.")
    def predictImage(self, image_input, result_count=5, input_type="file"):
        
        return self.classifyImage(image_input, result_count, input_type)

In [33]:
import pandas as pd
data_frame = pd.read_csv('/data.csv')
data_frame

from bokeh.plotting import figure,show
from bokeh.models import HoverTool

TOOLTIPS = HoverTool(tooltips=[
    ("index", "$index"),
    ("(Wage,Value)", "(@Wage, @Value)"),
    ("Name", "@Name")]
)

p = figure(title="Soccer 2019", x_axis_label='Wage', y_axis_label='Value', plot_width=700, plot_height=700, tools=[TOOLTIPS])
p.circle('Wage', 'Value', size=10, source=df1)
show(p)


In [31]:
import pandas as pd
data_frame = pd.read_csv('/data.csv')
data_frame
df1 = pd.DataFrame(data_frame, columns=['Name', 'Wage', 'Value'])
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

wage = df1['Wage'].replace('[\€,]', '', regex=True).apply(value_to_float)
value = df1['Value'].replace('[\€,]', '', regex=True).apply(value_to_float)

df1['Wage'] = wage
df1['Value'] = value

df1['difference'] = df1['Value'] - df1['Wage']
df1.sort_values('difference', ascending=False)


,Name,Wage,Value,difference
2,Neymar Jr,290000.0,118500000.0,118210000.0
0,L. Messi,565000.0,110500000.0,109935000.0
4,K. De Bruyne,355000.0,102000000.0,101645000.0
5,E. Hazard,340000.0,93000000.0,92660000.0
15,P. Dybala,205000.0,89000000.0,88795000.0
...,...,...,...,...
17752,S. Phillips,1000.0,0.0,-1000.0
12192,H. Sulaimani,3000.0,0.0,-3000.0
3550,S. Nakamura,4000.0,0.0,-4000.0
4228,B. Nivet,5000.0,0.0,-5000.0


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import joblib 
 
# 3. Load red wine data.
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')
 
# 4. Split data into training and test sets
y = data.quality
X = data.drop('quality', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)
 
# 5. Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(),
                         RandomForestRegressor(n_estimators=100,
                                               random_state=123))
 
# 6. Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}
 
# 7. Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
clf.fit(X_train, y_train)
 
# 8. Refit on the entire training set
# No additional code needed if clf.refit == True (default is True)
 
# 9. Evaluate model pipeline on test data
pred = clf.predict(X_test)
print( r2_score(y_test, pred) )
print( mean_squared_error(y_test, pred) )
 
# 10. Save model for future use
joblib.dump(clf, 'rf_regressor.pkl')
# To load: clf2 = joblib.load('rf_regressor.pkl')

0.4712595193413647
0.34118218749999996


['rf_regressor.pkl']

In [1]:
#1 - Import the data
#2 - Clean the data
#3 - Split data. Training Set/Test Set
#4 - Create a Model 
#5 - Check the output
#6 - Improve

In [ ]:
import requests
import hashlib


def request_api_data(query_char):
    url = 'https://api.pwnedpasswords.com/range/' + query_char
    res = requests.get(url)
    if res.status_code != 200:
        raise RuntimeError(f'Error fetching: {res.status_code} check the API')
    return res


def get_password_leaks_count(hashes, hash_to_check):
    hashes = (line.split(':') for line in hashes.text.splitlines())
    for h, count in hashes:
        if h == hash_to_check:
            return count
    return 0


def pwned_api_check(password):
    sha1password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    first5_char, tail = sha1password[:5], sha1password[5:]
    response = request_api_data(first5_char)
    return get_password_leaks_count(response, tail)


def main(args):
    count = pwned_api_check(args)
    if count:
        print(f'{args} was found {count} times')
    else:
        print(f'{args} was not found')


if __name__ == '__main__':
    while True:
        password = input('type "break" to exit\npassword: ')
        
        if password == 'break':
            break

        main(password)

type "break" to exit
password: shirin
shirin was found 7250 times
type "break" to exit
password: break


In [ ]:
def do_stuff(num=0):
  try:
    if num: # try to test for when input is 0. How can you fix this?
      return int(num) + 5
    else:
      return 'please enter number'
  except ValueError as err:
    return err

In [ ]:
import re

pattern = re.compile(r'[a-zA-Z0-9$%#@]{8,}\d$')
string = '123Asdaw@$4'
a = pattern.fullmatch(string)

if a == None:
    print('your password must be at least 8 digits long\n contain letters, numbers, any of these symbols $%#@ and end with a number')

else:
    print('logged in')

logged in


In [ ]:
password = "secret@#$%123"
pattern = re.compile(r"[a-zA-Z0-9@#$%]{8,}\d")

b = pattern.fullmatch(password)

print(password)
    
try:
    if b.group() != None:
        print("Password is valid.")
except AttributeError:
    print("Password does not meet criteria.")

print()

secret@#$%123
Password is valid.



In [ ]:
import re

pattern = re.compile(r"^[A-Za-z0-9#?!@$%^&*-].{8,}\d$")
string = 'jjw123fdafdasfdafa9'

a = pattern.fullmatch(string)

print(a)

<re.Match object; span=(0, 19), match='jjw123fdafdasfdafa9'>


In [ ]:
import re

txt = "The rain in Spain"
x = re.findall("ssshhhrrrrjfff", txt)
print(x)

[]


In [ ]:
import re

txt = "The rain in Spain"
x = re.findall("Coimbaore", txt)
print(x)

[]


In [ ]:
def derive(self, exc):
        return MyGroup(self.message, exc)
try:
   raise e
except Exception as e:
   exc = e

In [ ]:
some_list = ['a', 'b', 'c', 'b', 'd', 'm', 'n', 'n']
duplicates = list(set([x for x in some_list if some_list.count(x) > 1]))
print(duplicates)

['b', 'n']


In [ ]:
some_list = ['a', 'b', 'c', 'b', 'd', 'm', 'n', 'n']

duplicates = []
for value in some_list:
    if some_list.count(value) > 1:
        if value not in duplicates:
            duplicates.append(value)

print(duplicates)

['b', 'n']


In [ ]:
print(list(map(lambda num: num**2, [1,2,3])))

a = [(0, 2), (5, 2), (9, 9), (10, -1)]
a.sort(key=lambda x: x[1])

print(a)

[1, 4, 9]
[(10, -1), (0, 2), (5, 2), (9, 9)]


In [ ]:
from functools import reduce
my_pets = ['sisi', 'bibi', 'titi', 'carla']

def capitalize(string):
    return string.upper()

print(list(map(capitalize, my_pets)))

my_strings = ['a', 'b', 'c', 'd', 'e']
my_numbers = [5,4,3,2,1]

print(list(zip(my_strings, sorted(my_numbers))))

scores = [73, 20, 65, 19, 76, 100, 88]

def is_smart_student(score):
    return score > 50

print(list(filter(is_smart_student, scores)))

def accumulator(acc, item):
    return acc + item

print(reduce(accumulator, (my_numbers + scores)))

['SISI', 'BIBI', 'TITI', 'CARLA']
[('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)]
[73, 65, 76, 100, 88]
456


In [ ]:
from functools import reduce
my_pets = ['sisi', 'bibi', 'titi', 'carla']

my_strings = ['a', 'b', 'c', 'd', 'e']
my_numbers = [5,4,3,2,1]
scores = [73, 20, 65, 19, 76, 100, 88]


In [ ]:
class A:
    num = 10


class B(A):
    pass

class C(A):
    num = 1

class D(B, C):
    pass
print(A)
print(D.mro())


<class '__main__.A'>
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


In [ ]:
class SuperList(list):
  def __len__(self):
    return 1000

super_list1 = SuperList();

print(len(super_list1))
super_list1.append(5)
print(super_list1[0])
print(issubclass(list, object))

1000
5
True


In [ ]:
class Toy():
  def __init__(self, color, age):
    self.color = color
    self.age = age
    self.my_dict = {
        'name':'Yoyo',
        'has_pets': False,
    }

  def __str__(self):
    return f"{self.color}"

  def __len__(self):
    return 5

  def __del__(self):
    return "deleted"

  def __call__(self):
      return('yes??')

  def __getitem__(self,i):
      return self.my_dict[i]


action_figure = Toy('red', 0)
print(action_figure.__str__())
print(str(action_figure))
print(len(action_figure))
print(action_figure())
print(action_figure['name'])

red
red
5
yes??
Yoyo


In [ ]:
from inspect import isclass

def subscribe(obj, x):
    """Return the result of the expression 'obj[x]'"""

    class_of_obj = type(obj)

    
    if hasattr(class_of_obj, '__getitem__'):
        return class_of_obj.__getitem__(obj, x)

    
    elif isclass(obj) and hasattr(obj, '__class_getitem__'):
        return obj.__class_getitem__(x)

    
    else:
        raise TypeError(
            f"'{class_of_obj.__name__}' object is not subscriptable"
        )

In [ ]:
class Toy():
  def __init__(self, color, age):
    self.color = color
    self.age = age
    self.my_dict = {
        'name':'Yoyo',
        'has_pets': False,
    }

  def __str__(self):
    return f"{self.color}"

  def __len__(self):
    return 5

  def __del__(self):
    return "deleted"

  def __call__(self):
      return('yes??')

  def __getitem__(self,i):
      return self.my_dict[i]


action_figure = Toy('red', 0)
print(action_figure.__str__())
print(str(action_figure))
print(len(action_figure))
print(action_figure())
print(action_figure['name'])

red
red
5
yes??
Yoyo


In [ ]:
class Pets():
    animals = []
    def __init__(self, animals):
        self.animals = animals

    def walk(self):
        for animal in self.animals:
            print(animal.walk())

class Cat():
    is_lazy = True

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def walk(self):
        return f'{self.name} is just walking around'

class Simon(Cat):
    def sing(self, sounds):
        return f'{sounds}'

class Sally(Cat):
    def sing(self, sounds):
        return f'{sounds}'

In [ ]:
class PlayerCharacter:
  membership = True
  def __init__(self, name, age):
    self.name = name
    self.age = age
  def run(self):
    print('run')

In [ ]:
class PlayerCharacter:
  membership = True
  def __init__(self, name, age):
    self.name = name
    self.age = age
  def shout(self):
    print('run')
  @classmethod
  def adding_things(cls, num1, num2):
    return num1 + num2
  def PlayerCharacter():
    player1 = PlayerCharacter('tom', 1)
    print(player1.adding_things(2,3))

In [ ]:
class Cat:
    species = 'mammal'
    def __init__(self, name, age):
        self.name = name
        self.age = age


In [ ]:
class PlayerCharacter:
  def __init__(self, name):
    self.name = name
  def run(self):
    print('run')

player1 = PlayerCharacter("shi")

print(player1)

In [ ]:
class BigObject:
    pass

obj1 = BigObject()
obj2 = BigObject()
obj3 = BigObject()

print(type(None))
print(type(True))
print(type(5))
print(type(5.5))
print(type('hi'))
print(type([]))
print(type(()))
print(type({}))
print(type(obj1))

<class 'NoneType'>
<class 'bool'>
<class 'int'>
<class 'float'>
<class 'str'>
<class 'list'>
<class 'tuple'>
<class 'dict'>
<class '__main__.BigObject'>


In [ ]:
print(type(None))
print(type(True))
print(type(5))
print(type(5.5))
print(type('hi'))
print(type([]))
print(type(()))
print(type({}))

<class 'NoneType'>
<class 'bool'>
<class 'int'>
<class 'float'>
<class 'str'>
<class 'list'>
<class 'tuple'>
<class 'dict'>


In [ ]:
def outer():
  x = "local"
  def inner():
    nonlocal x
  x = "nonlocal"
  print("inner:", x)
  inner()
print("outer:", x)
outer()


outer: 10
inner: nonlocal


In [ ]:
total = 10
def count(total):
   total += 1
   return total
print(count(count(count(total))))

13


In [ ]:
total = 100
def count(total):
   total += 1
   return total
print(count(count(count(total))))

103


In [ ]:
a = 10
def confusion(b):
  print(b)
  a = 90
confusion(300)

300


In [ ]:
if True:
  x = 10
def some_function():
  total = 100
print(x)

10


In [ ]:
a = 'helllloooooooooooooooooooooo'
if((n := len(a)) > 10):
  print(f"too long {n} elements")
while ((n := len(a)) > 1):
  print(n)
  a = a[:-1]
print(a)

too long 28 elements
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
h


In [ ]:
def test(a):
  '''
  Info : this function tests and prints param
  '''
  print(a)


help(test.__doc__)

No Python documentation found for 'Info : this function tests and prints param'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



In [ ]:
def test(a):
  '''
  Info : this function tests and prints param
  '''
  print(a)


help(test)


Help on function test in module __main__:

test(a)
    Info : this function tests and prints param



In [ ]:
def checkDriverAge(age=0):
    if int(age) < 18:
        print("Sorry, you are too yound to drive this car. Powering off")
    elif int(age) > 18:
        print("Powering On. Enjoy the ride!");
    elif int(age) == 18:
        print("Congratulations on your first year of driving. Enjoy the ride!")
checkDriverAge()

Sorry, you are too yound to drive this car. Powering off


In [ ]:
def checkDriverAge():
    age = input("What is your age?: ")
    if int(age) < 18:
        print("Sorry, you are too young to drive this car. Powering off")
    elif int(age) > 18:
        print("Powering On. Enjoy the ride!");
    elif int(age) == 18:
        print("Congratulations on your first year of driving. Enjoy the ride!")
checkDriverAge()

What is your age?: 22
Powering On. Enjoy the ride!


In [ ]:

def sum(num1, num2):
  return num1 + num2
print(sum(4,5))

9


In [ ]:
picture = [
[0,0,0,1,0,0,0],
[0,0,1,1,1,0,0],
[0,1,1,1,1,1,0],
[1,1,1,1,1,1,1],
[0,0,0,1,0,0,0],
[0,0,0,1,0,0,0]
]
def show_tree():
  for image in picture:
    for pixel in image:
      if(pixel):
        print('*', end = "")
  else:
    print('', end = "")
  print('')

In [ ]:
some_list = ['a', 'b', 'c', 'b', 'd', 'm', 'n', 'n']
duplicate = []
for value in some_list:
  if some_list.count(value) > 1:
    duplicate.append(value)
  print(duplicate)

[]
['b']
['b']
['b', 'b']
['b', 'b']
['b', 'b']
['b', 'b', 'n']
['b', 'b', 'n', 'n']


In [ ]:
picture = [
[0,0,0,1,0,0,0],
[0,0,1,1,1,0,0],
[0,1,1,1,1,1,0],
[1,1,1,1,1,1,1],
[0,0,0,1,0,0,0],
[0,0,0,1,0,0,0]
]
for image in picture:
  for pixel in image:
    if (pixel):
      print('*', end ="")
    else:
      print(' ', end ="")
  print('')


   *   
  ***  
 ***** 
*******
   *   
   *   


In [ ]:
my_list = [1,2,3]
for item in my_list:
  pass
  print(item)

1
2
3


In [ ]:
my_list = [1,2,3]
for item in my_list:
  print(item)
  continue

1
2
3


In [ ]:
my_list = [1,2,3]
for item in my_list:
  print(item)

i = 0
while i < len(my_list):
  print(my_list[i])
  i += 1

1
2
3
1
2
3


In [ ]:
i = 0
while i < 50:
  print(i)
  i += 1
else:
  print('done with all the work')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
done with all the work


In [ ]:
i = 0
while i < 50:
  print(i)
  break

0


In [ ]:
for i, char in enumerate('hi'):
  print(i, char)

0 h
1 i


In [ ]:
for number in range(0,100):
  print(number)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
user = {
    'name' : "rafia",
    'age': 22,
    'can_swim' : False

}
for item in user: 
    print(item)

name
age
can_swim


In [ ]:
for item in 'Zero to Mastery':
  print(item)
for item2 in 1,2,3:
  print(item2)

Z
e
r
o
 
t
o
 
M
a
s
t
e
r
y
1
2
3


In [ ]:
print(True == 1)
print("" == 1)
print([] == 1)
print(10 == 10.0)
print([] == [])


True
False
False
True
True


In [ ]:
is_magician = True
is_expert = False
if is_magician and is_expert:
  print('you are a master magician')
elif is_magician and not is_expert:
  print("atleast you are getting there")
elif not is_magician:
  print("you need magic powers")

atleast you are getting there


In [ ]:
print(1<2<3<4)

True


In [ ]:
print(1<2>3>4)

False


In [ ]:
is_Friend = True
is_User = False
if is_Friend or is_User:
  print('best friends forever')

best friends forever


In [ ]:
is_friend = True
can_message = "message allowed" if is_friend else "not allowed to message"
print(can_message)

message allowed


In [ ]:
is_old = bool('hello')
is_licensed = bool(5)
print(bool('hello'))
print(bool(5))

if is_old:
  print('you are old enough to drive!')
else:
  print('checkcheck')

True
True
you are old enough to drive!


In [ ]:
is_old = True
is_licensed = True

if is_old:
  print('you are old enough to drive!')
else:
  print('checkcheck')

you are old enough to drive!


In [ ]:
is_old = True
is_licensed = True

if is_old:
  print('you are old enough to drive!')
  print('checkcheck')

you are old enough to drive!
checkcheck


In [ ]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.union(your_set))


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [ ]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.intersection(your_set))


{4, 5}


In [ ]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.difference(your_set))
print(my_set)

{1, 2, 3}
{1, 2, 3, 4, 5}


In [ ]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.discard(5))
print(my_set)

None
{1, 2, 3, 4}


In [ ]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.difference(your_set))

{1, 2, 3}


In [ ]:
my_set = {1,2,3,4,5}
my_set.add(100)
my_set.add(2)
print(my_set)

{1, 2, 3, 4, 5, 100}


In [ ]:
my_tuple = (1, 2, 3, 4, 5)
new_tuple = my_tuple[1:4]
print(new_tuple)

(2, 3, 4)


In [ ]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print('hello' in user.keys())

False


In [ ]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print('hello' in user.values())

True


In [ ]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print(user.get('age'))

20


In [ ]:
a,b,c, * other, d = [1,2,3,4,5,6,7,8,9]
print(a)
print(b)
print(c)
print(other)
print(d)

1
2
3
[4, 5, 6, 7, 8]
9


In [ ]:
dict = {'a':1, 'b' : 'hellloo', 'c': True}
print(dict['c'])

True


In [ ]:
dict = {'a':1, 'b' : 2, 'c':7}
print(dict)

{'a': 1, 'b': 2, 'c': 7}


In [ ]:
basket = ['a','b','z','v','d']
basket.sort()
basket.reverse()
print(basket[::1])

['z', 'v', 'd', 'b', 'a']


In [ ]:
basket.pop()
print(basket)

[1, 2, 3, 4, 5]


In [ ]:
basket = ['a','b','c']
print(basket.index('b'))

1


In [ ]:
basket = ['a','b','z','v','d']
basket.sort()
print(basket)

['a', 'b', 'd', 'v', 'z']


In [ ]:
basket = [1,2,3,4,5]
new_list = basket.append(100)
print(basket)
print(new_list)

[1, 2, 3, 4, 5, 100]
None


In [ ]:
amazon_cart = ['notebooks', 'sunglasses', 'toys', 'bicycles']
print(amazon_cart[0:2])

['notebooks', 'sunglasses']


In [ ]:
amazon_cart[0] = 'laptops'
print(amazon_cart)

['laptops', 'sunglasses', 'toys', 'bicycles']


In [ ]:
matrix = [[0,1,2],[3,4,5],[6,7,8]]

print(matrix[0][2])

2


In [ ]:
basket = [1,2,3]
print(len(basket))

3


In [ ]:
username = input('what is your name?')
password = input('what is your password?')

password_length = len(password)
hidden_password = '*' * password_length

print(f'{username}, your password, {hidden_password}, is {password_length} letters long')

what is your name?shirin
what is your password?Shirinrafia*123
shirin, your password, ***************, is 15 letters long


# **HighIQ Python Trainer Notebook** *(Zero to One)*

In [ ]:
amazon_cart = ['notebooks', 'sunglasses']
print(amazon_cart[0])

notebooks


In [ ]:
birth_year = input('what year you were born?')

age = 2022 - int(birth_year)

print(f'your age is : {age} ')

what year you were born?2000
your age is : 22 


In [ ]:
name = 'shirin'
is_nice = False
is_nice = True

print(bool(0))

False


In [ ]:
selfish = 'me me me'
print(selfish[6]) 


m


In [ ]:
weather = "It's \"kind of\"sunny"
print(weather)

It's "kind of"sunny


In [ ]:
a = str(100)
b = int(a)
c = type(b)
print(c)

<class 'int'>


In [ ]:
print(type(int(str(100))))

<class 'int'>


In [ ]:
print('hey '  +  'shirin')

hey shirin


In [ ]:
print(type('hi shirin'))
long_string = '''
WOW
0  0
----
'''
print(long_string)

<class 'str'>

WOW
0  0
----



In [ ]:
iq = 100
user_age = iq/5

In [ ]:
some_value = 5
some_value = some_value + 6

print(some_value)

11


# [**Click on this Course**](https://www.youtube.com/watch?v=eWRfhZUzrAc&t=2s)

In [ ]:
print(bin(5))

0b101


In [ ]:
input('what is your age?')

what is your age?22


'22'

In [ ]:
print("shirinrafia")

shirinrafia


## Variable And Functions

## Calling Funtions

# Dictionaries

## User Input

## Libraries, Lists, Methods

## Function Arguments

## If Statements 

## Concatenating Strings

## f-strings

## Else and Elif Statements

## Refactoring and Nested If

## Accessing Dictionary Values

## Testing Game

## Variables

## Expressions and Statements 

## Comments

## Data Types

## Operators

## Arithmetic Operators

## Comparison Operators

## Boolean Operators

## Bitwise Operators

## is & in Operators

## Ternary Operator

## Strings

## String Methods

## Escaping Characters

## String Characters & Slicing

## Booleans

## Number Data Types

## Built-in Functions

In [ ]:
print(round(-5.5))

-6


## Enums

In [ ]:
from enum import Enum

class State(Enum):
  INACTIVE = 0
  ACTIVE = 1

print(State.ACTIVE)

State.ACTIVE


## User Input

In [ ]:
print("What is your age? ")
age = input()
print("your age is  "  + age)


What is your age? 
22
your age is  22


## Control Statements

In [ ]:
condition = False

if condition == True:
  print()


## Lists

In [ ]:
dogs = ["Shirin", 1, "Rafia", True]

print("Shirin" in dogs)

True


## Sorting Lists

In [ ]:
from typing import ItemsView
items = ["shirin", "rafia", "Beau", "Quincy"]
itemscopy = items[:]
items.sort(key=str.lower)

print(items)
print(itemscopy)

['Beau', 'Quincy', 'rafia', 'shirin']
['shirin', 'rafia', 'Beau', 'Quincy']


## Tuples

In [ ]:
names = ("rogger", "aadhil")

names[-1]
names.index("rogger")

len(names)

print("rogger" in names)

True


## Dictionaries

In [ ]:
dog = {"name": "Rafia", "age": 8}

print(dog['name'])

Rafia


## Sets

In [ ]:
set1 = {"rafia", "shirin"}
set2 = {"rogger"}

intersect = set1 & set2
print(intersect)

set()


## Functions

In [ ]:
def hello(name):
  print('Hello ' + name)

  hello("shirin")
  hello("thaaka")

## Variable Scope

In [ ]:
  def test():
    age = 8
    print(age)


## Nested Functions 

In [ ]:
def talk(phrase):
  def say(word):
    print(word)

    words = phrase.split(' ')
    for word in words:
      say(word)

      talk('I am going to buy milk')


## Closures

In [ ]:
def counter():
  count = 0

  def increment():
    nonlocal count
    count = count + 1
    return count

    return increment
    increment = counter()

    print(increment()) #1
    print(increment()) #2
    print(increment()) #3


## Objects

In [ ]:
age = 8

print(age.real)

8


## Loops

In [ ]:
condition = True
while condition == True:
  print("The condition is True")
  condition = False

The condition is True


## Break and Continue

In [ ]:
items = [1, 2, 3, 4]
for item in items:
  if item == 2:
    continue
    print(item)

## Classes

In [ ]:
class Dog:
  def bark(self):
    print("woof!")

## Modules

In [ ]:
def bark():
  print("woof!")

## Arguments from Command Line

In [ ]:
print("hello")

hello


## Lambda Functions

In [ ]:
lambda num : num * 2

<function __main__.<lambda>(num)>

## Map, Filter, Reduce

In [ ]:
numbers = [1, 2, 3]

def double(a):
  return a * 2

  result = map(double, numbers)

  def fun(variable):
    letters = ['a', 'e', 'i', 'o', 'u']
    if (variable in letters):
        return True
    else:
        return False

## Recursion 

In [ ]:
def factorial(x):


    if x == 1:
        return 1
    else:
        return (x * factorial(x-1))


num = 3
print("The factorial of", num, "is", factorial(num))

The factorial of 3 is 6


# The remaining modules from the Video is Optional if you wish to continue you can continue

In [ ]:
def first(msg):
    print(msg)


first("Hello")

second = first
second("Hello")

Hello
Hello
